In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3
...,...,...,...,...,...,...,...,...,...
20973,23/06/2024 19:00,39.3,92,155.6,21.0,105.3,57.0,31.33,17.0
20974,23/06/2024 20:00,37.0,98,191.4,30.0,80.0,58.0,34.00,20.0
20975,23/06/2024 21:00,35.6,98,233.6,39.7,60.7,58.0,34.33,23.7
20976,23/06/2024 22:00,34.4,99,275.8,49.3,41.3,58.0,34.67,27.3


In [5]:
temp = df[['temp']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = temp.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [8]:
scaled = scaled.reshape((20978,))

In [9]:
scaled.shape

(20978,)

In [10]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [42]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [44]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [46]:
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [47]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)                    │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0235 - root_mean_squared_error: 0.1391 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0692 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0051 - root_mean_squared_error: 0.0717 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0688 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0050 - root_mean_squared_error: 0.0705 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0549 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0050 - root_mean_squared_error: 0.0707 - val_loss: 0.0032 - val_root_mean_squared_error: 0.0561 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0048 - root_mean_squared_error: 0.0695 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0545 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0049 - root_mean_squared_err

In [48]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [79]:
poll = np.array(df["temp"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [80]:
from sklearn.metrics import mean_squared_error

In [81]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 0.5889489


In [82]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 1.6034122556447983


In [83]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 98.3965877443552 %


In [84]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 0.45972964


In [55]:
forecast

array([[0.18514118, 0.16574669, 0.16048098, ..., 0.24517705, 0.23057435,
        0.19898479],
       [0.16213939, 0.14649433, 0.13913143, ..., 0.23390768, 0.20005587,
        0.20180704],
       [0.13847683, 0.13747695, 0.13142288, ..., 0.20114364, 0.19543317,
        0.17979053],
       ...,
       [0.8256635 , 0.7727042 , 0.76187307, ..., 0.82369876, 0.7613567 ,
        0.74605733],
       [0.77712834, 0.74925584, 0.7426303 , ..., 0.7591944 , 0.749273  ,
        0.7260011 ],
       [0.764101  , 0.7348347 , 0.701817  , ..., 0.7444183 , 0.7353683 ,
        0.6802917 ]], dtype=float32)

In [56]:
testPredict

array([[26.225372, 26.052591, 26.00568 , ..., 26.760216, 26.630123,
        26.348701],
       [26.020454, 25.881077, 25.815483, ..., 26.65982 , 26.358242,
        26.373844],
       [25.809652, 25.800743, 25.746809, ..., 26.367933, 26.31706 ,
        26.177704],
       ...,
       [31.931614, 31.459814, 31.363323, ..., 31.914112, 31.358723,
        31.222424],
       [31.499228, 31.25092 , 31.191895, ..., 31.339458, 31.251072,
        31.043749],
       [31.383171, 31.122444, 30.828299, ..., 31.207823, 31.1272  ,
        30.636536]], dtype=float32)

In [57]:
y_test_true

array([[26.242529, 26.16023 , 26.016209, ..., 25.625296, 25.707592,
        25.563572],
       [26.16023 , 26.016209, 26.098507, ..., 25.707592, 25.563572,
        25.625296],
       [26.016209, 26.098507, 25.933912, ..., 25.563572, 25.625296,
        25.707592],
       ...,
       [31.735905, 31.283266, 31.283266, ..., 32.27084 , 32.04452 ,
        31.571308],
       [31.283266, 31.283266, 31.20097 , ..., 32.04452 , 31.571308,
        31.283266],
       [31.283266, 31.20097 , 30.892353, ..., 31.571308, 31.283266,
        31.036373]], dtype=float32)

In [58]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [59]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.2355
RMSE for segment 2: 0.2751
RMSE for segment 3: 0.3002
RMSE for segment 4: 0.3286
RMSE for segment 5: 0.3525
RMSE for segment 6: 0.3676
RMSE for segment 7: 0.3793
RMSE for segment 8: 0.3903
RMSE for segment 9: 0.4009
RMSE for segment 10: 0.4050
RMSE for segment 11: 0.4098
RMSE for segment 12: 0.4151
RMSE for segment 13: 0.4155
RMSE for segment 14: 0.4176
RMSE for segment 15: 0.4135
RMSE for segment 16: 0.4129
RMSE for segment 17: 0.4127
RMSE for segment 18: 0.4122
RMSE for segment 19: 0.4173
RMSE for segment 20: 0.4119
RMSE for segment 21: 0.4108
RMSE for segment 22: 0.4113
RMSE for segment 23: 0.4131
RMSE for segment 24: 0.4155
RMSE for segment 25: 0.4243
RMSE for segment 26: 0.4350
RMSE for segment 27: 0.4396
RMSE for segment 28: 0.4535
RMSE for segment 29: 0.4629
RMSE for segment 30: 0.4700
RMSE for segment 31: 0.4789
RMSE for segment 32: 0.4839
RMSE for segment 33: 0.4922
RMSE for segment 34: 0.4944
RMSE for segment 35: 0.4996
RMSE for segment 36: 0.5016
R

In [85]:
model.save("temp_1dcnn.keras")